In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


df = pd.read_csv("data.csv")

df['Email'] = df['Email'].fillna(df['Name'].str.lower() + "@gmail.com")

df['Age'] = df['Age'].fillna(df['Age'].mean())

df['City'] = df['City'].fillna(df['City'].mode()[0])

df['Experience'] = df['Experience'].fillna(method='ffill')


#Encoding categorical values
promo_le = LabelEncoder()
df_label_encode = df.copy()

df_label_encode['Promotion'] = promo_le.fit_transform(df_label_encode['Promotion'])

#One hot coding
one_hot_encoding = pd.get_dummies(df_label_encode[['Department', 'City']])


#Concating them
df_final = pd.concat([
    df_label_encode[['Age','Experience','Salary','Performance']], 
    one_hot_encoding,                                             
    df_label_encode['Promotion']                           
], axis=1)
print(df_final.head(5))

#Scaling:
scale = MinMaxScaler()
numeric_feature = ['Age', 'Experience', 'Salary', 'Performance']
df_final[numeric_feature] = scale.fit_transform(df_final[numeric_feature])

#traintestsplit

X = df_final.drop('Promotion', axis=1)
y = df_final['Promotion']

X_test, X_train, y_test, y_train = train_test_split(
    X, y, test_size = 0.3, random_state = 42
)


log_model = LogisticRegression()
log_model.fit(X_train, y_train)

y_pred = log_model.predict(X_test)

#For accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Logistic Regression Accuracy:", accuracy)

new_employee = [[0.5, 0.2, 0.4, 0.8] + [0]*(X.shape[1]-4)]  # scaled numeric + zeros for one-hot
promotion_pred = log_model.predict(new_employee)[0]
if promotion_pred == 1:
    print("The new employee is likely to get promoted ")
else:
    print("The new employee is unlikely to get promoted ")









         Age  Experience  Salary  Performance  Department_Finance  \
0  23.000000         1.0   18000            2               False   
1  27.000000         4.0   30000            4               False   
2  27.642857         3.0   26000            3                True   
3  29.000000         5.0   40000            5               False   
4  31.000000         7.0   55000            5               False   

   Department_HR  Department_IT  Department_Marketing  City_Butwal  \
0          False           True                 False        False   
1           True          False                 False        False   
2          False          False                 False         True   
3          False           True                 False        False   
4          False          False                  True        False   

   City_Kathmandu  City_Lalitpur  City_Pokhara  Promotion  
0            True          False         False          0  
1           False          False          Tr

C:\Users\subin\AppData\Local\Temp\ipykernel_24584\3012416887.py:16: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Experience'] = df['Experience'].fillna(method='ffill')
c:\Users\subin\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
